# CSI 4142 Project Deliverable 3
## OLAP Queries, and BI Dashboard
### Group 1 Member
| Name | Student ID |
|-------|---------------|
| Gary Gao | 300124236|
| Yingqi Feng | 300077437|
| Binxuan Wu | 300142301 | 

In [ ]:
import numpy as np
import pandas as pd

##Loading Data

In [ ]:
df_building =  pd.read_csv("https://raw.githubusercontent.com/BinxuanWu/CSI4142-Project/main/domain%20graphs/building.csv", encoding = 'unicode_escape', dtype={0:str})
df_community =  pd.read_csv("https://raw.githubusercontent.com/BinxuanWu/CSI4142-Project/main/domain%20graphs/community.csv", encoding = 'unicode_escape')
df_date =  pd.read_csv("https://raw.githubusercontent.com/BinxuanWu/CSI4142-Project/main/domain%20graphs/date.csv", encoding = 'unicode_escape')
df_house =  pd.read_csv("https://raw.githubusercontent.com/BinxuanWu/CSI4142-Project/main/domain%20graphs/housing_price.csv", encoding = 'unicode_escape')

In [ ]:
df_building

,id,square,living_room,drawing_room,kitchen,bath_room,price,total_price,floor,floor_level,building_type,construction_time,building_structure,ladder_ratio,five_years_property,elevator,renovation_condition
0,101084782030,131.00,2,1,1,1,31680,4150000.0,26,high,1.0,2005,6,0.217,0,1,3
1,101086012217,132.38,2,2,1,2,43436,5750000.0,22,high,1.0,2004,6,0.667,1,1,4
2,101086041636,198.00,3,2,1,3,52021,10300000.0,4,middle,4.0,2005,6,0.500,0,1,3
3,101086406841,134.00,3,1,1,1,22202,2975000.0,21,bottom,1.0,2008,6,0.273,0,1,1
4,101086920653,81.00,2,1,1,1,48396,3920000.0,6,middle,4.0,1960,2,0.333,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318814,BJYZ92311192,170.07,3,2,1,2,20580,3500000.0,5,middle,4.0,2003,6,0.333,1,0,1
318815,BJYZ92320171,35.09,1,0,1,1,31006,1088000.0,24,middle,1.0,2009,6,0.222,0,1,1
318816,BJYZ92324217,102.17,2,2,1,2,35138,3590000.0,7,middle,4.0,2000,6,0.500,1,1,3
318817,BJYZ92333313,178.34,4,2,1,2,40373,7200000.0,5,middle,4.0,2003,2,0.500,1,0,4


In [ ]:
df_community.head()

,cid,district,subway,community_average
0,1111027373683,1,1,95763
1,1111027373684,1,1,87076
2,1111027373685,1,1,86883
3,1111027373686,7,0,82664
4,1111027373689,1,1,94481
...,...,...,...,...
4030,117258115420294,1,0,139053
4031,117800835289894,6,1,53998
4032,118372281946889,6,1,50274
4033,118486728773575,6,0,65970


In [ ]:
df_date.head()

,trade_time,trade_year,trade_month
0,2002-06-01,2002,6
1,2002-07-06,2002,7
2,2002-07-10,2002,7
3,2003-07-31,2003,7
4,2008-10-07,2008,10
...,...,...,...
2549,2018-01-20,2018,1
2550,2018-01-22,2018,1
2551,2018-01-24,2018,1
2552,2018-01-26,2018,1


In [ ]:
df_house.head()

,surrogate_keys,id,cid,trade_time,price
0,1,101084782030,1111027376244,2016-08-09,31680
1,2,101086012217,1111027381879,2016-07-28,43436
2,3,101086041636,1111040862969,2016-12-11,52021
3,4,101086406841,1111043185817,2016-09-30,22202
4,5,101086920653,1111027381174,2016-08-28,48396
...,...,...,...,...,...
318814,318815,BJYZ92311192,1111027377053,2016-06-10,20580
318815,318816,BJYZ92320171,1111027377957,2016-06-10,31006
318816,318817,BJYZ92324217,1111027380056,2016-06-05,35138
318817,318818,BJYZ92333313,1111027377054,2016-06-12,40373


**Merged the 3 different tables based on FK and PK**

In [ ]:
df_temp = pd.merge(df_house, df_building.drop('price', axis = 1), how='inner', left_on = 'id', right_on = 'id')
df_com_bulid = pd.merge(df_temp, df_community, how='inner', left_on = 'cid', right_on = 'cid')
df_com_bulid

,surrogate_keys,id,cid,trade_time,price,square,living_room,drawing_room,kitchen,bath_room,...,building_type,construction_time,building_structure,ladder_ratio,five_years_property,elevator,renovation_condition,district,subway,community_average
0,1,101084782030,1111027376244,2016-08-09,31680,131.00,2,1,1,1,...,1.0,2005,6,0.217,0,1,3,7,1,56021
1,2581,101091790306,1111027376244,2016-06-29,43629,49.28,1,0,1,1,...,1.0,2005,6,0.217,0,1,4,7,1,56021
2,3140,101091848103,1111027376244,2016-09-06,42101,85.51,1,1,1,1,...,1.0,2005,6,0.217,1,1,1,7,1,56021
3,3198,101091852185,1111027376244,2016-09-18,42186,99.56,1,1,1,1,...,1.0,2005,6,0.217,0,1,1,7,1,56021
4,8879,101092246232,1111027376244,2016-09-08,51680,65.21,1,0,1,1,...,1.0,2005,6,0.217,1,1,4,7,1,56021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318814,317023,BJXC92150717,1111027373965,2016-05-23,149442,14.32,1,0,0,0,...,2.0,1914,3,1.000,0,0,3,10,0,59015
318815,317059,BJXC92185193,1111047812347,2016-05-24,113247,38.50,2,0,0,0,...,2.0,0,1,1.000,0,0,3,10,1,130768
318816,317102,BJXC92215207,1111027378919,2016-05-22,145981,15.55,1,0,0,0,...,0.0,0,3,0.250,1,0,3,10,1,59015
318817,317155,BJXC92255534,1111027381192,2016-05-25,49987,38.36,1,1,1,1,...,0.0,0,3,0.125,1,0,3,10,1,59015


# Part A.1. Standard OLAP operations

## Rollup and Drilldown

**Rollup and find the average price of each district**




In [ ]:
rollup_df = df_com_bulid.groupby(['district']).agg({'price':'mean'}).reset_index()
rollup_df

,district,price
0,1,62024.151235
1,2,38179.134627
2,3,31312.978715
3,4,30024.005355
4,5,28329.428088
5,6,29380.591153
6,7,43628.866173
7,8,54856.529256
8,9,35171.638642
9,10,67615.931582


**Drill down and find the averagr price of house entries in each cid**


In [ ]:
drilldown_df = df_com_bulid.groupby(['district','cid']).agg({'price':'mean'}).reset_index()
drilldown_df

,district,cid,price
0,1,1111027373683,63041.270833
1,1,1111027373684,56733.562500
2,1,1111027373685,55955.442748
3,1,1111027373689,57058.272727
4,1,1111027373698,75168.333333
...,...,...,...
4030,13,1111027382152,25549.475480
4031,13,1111027382348,34702.477612
4032,13,1111041170735,40955.019231
4033,13,1111046920919,22909.702703


**Roll up and find the total number of house trade in each district**



In [ ]:
rollup_df2 = df_com_bulid.groupby(['district']).agg({'id':'count'}).reset_index().rename(columns={'id': 'building_count'})
rollup_df2

,district,building_count
0,1,17086
1,2,29333
2,3,2537
3,4,15312
4,5,2955
5,6,38633
6,7,107243
7,8,38197
8,9,11371
9,10,31293


**Drill down and find the number of house trade in each cid**

In [ ]:
drilldown_df2 = df_com_bulid.groupby(['district','cid']).agg({'id':'count'}).reset_index().rename(columns={'id': 'building_count'})

drilldown_df2

,district,cid,building_count
0,1,1111027373683,48
1,1,1111027373684,48
2,1,1111027373685,131
3,1,1111027373689,22
4,1,1111027373698,3
...,...,...,...
4030,13,1111027382152,469
4031,13,1111027382348,67
4032,13,1111041170735,104
4033,13,1111046920919,37


##Slice

slice by district 

In [ ]:
slice_df_1 = df_com_bulid.loc[df_com_bulid['district']==7]
slice_df_1

,surrogate_keys,id,cid,trade_time,price,square,living_room,drawing_room,kitchen,bath_room,...,building_type,construction_time,building_structure,ladder_ratio,five_years_property,elevator,renovation_condition,district,subway,community_average
0,1,101084782030,1111027376244,2016-08-09,31680,131.00,2,1,1,1,...,1.0,2005,6,0.217,0,1,3,7,1,56021
1,2581,101091790306,1111027376244,2016-06-29,43629,49.28,1,0,1,1,...,1.0,2005,6,0.217,0,1,4,7,1,56021
2,3140,101091848103,1111027376244,2016-09-06,42101,85.51,1,1,1,1,...,1.0,2005,6,0.217,1,1,1,7,1,56021
3,3198,101091852185,1111027376244,2016-09-18,42186,99.56,1,1,1,1,...,1.0,2005,6,0.217,0,1,1,7,1,56021
4,8879,101092246232,1111027376244,2016-09-08,51680,65.21,1,0,1,1,...,1.0,2005,6,0.217,1,1,4,7,1,56021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318143,241831,BJHD84202268,1111041783447,2012-02-06,19532,35.84,1,1,1,1,...,4.0,1982,4,0.333,1,0,1,7,0,51534
318144,249404,BJHD86236063,1111041783447,2013-08-24,33215,53.29,2,1,1,1,...,4.0,1982,4,0.333,1,0,1,7,0,51534
318470,284801,BJTJ84408045,1111027375997,2012-04-08,13861,57.72,2,1,1,1,...,4.0,1992,4,0.500,0,0,1,7,1,40498
318471,287102,BJTJ85606206,1111027375997,2013-03-15,11176,44.74,1,1,1,1,...,4.0,1988,4,0.333,1,0,1,7,1,40498


Slice by finding all the housing entries with building type 1



In [ ]:
slice_df_2 = df_com_bulid.loc[df_com_bulid['building_type']==1]
slice_df_2

,surrogate_keys,id,cid,trade_time,price,square,living_room,drawing_room,kitchen,bath_room,...,building_type,construction_time,building_structure,ladder_ratio,five_years_property,elevator,renovation_condition,district,subway,community_average
0,1,101084782030,1111027376244,2016-08-09,31680,131.00,2,1,1,1,...,1.0,2005,6,0.217,0,1,3,7,1,56021
1,2581,101091790306,1111027376244,2016-06-29,43629,49.28,1,0,1,1,...,1.0,2005,6,0.217,0,1,4,7,1,56021
2,3140,101091848103,1111027376244,2016-09-06,42101,85.51,1,1,1,1,...,1.0,2005,6,0.217,1,1,1,7,1,56021
3,3198,101091852185,1111027376244,2016-09-18,42186,99.56,1,1,1,1,...,1.0,2005,6,0.217,0,1,1,7,1,56021
4,8879,101092246232,1111027376244,2016-09-08,51680,65.21,1,0,1,1,...,1.0,2005,6,0.217,1,1,4,7,1,56021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318691,299368,BJXC85438594,1111027380977,2013-02-01,31918,112.79,2,2,1,1,...,1.0,2000,6,0.286,1,1,1,10,1,75358
318692,299616,BJXC85511103,1111027380977,2013-06-14,46014,96.71,2,2,1,1,...,1.0,2000,6,0.286,1,1,1,10,1,75358
318693,308552,BJXC89077302,1111027380977,2015-05-08,33176,122.38,2,1,1,1,...,1.0,2000,6,0.286,1,1,4,10,1,75358
318694,313037,BJXC90591689,1111027380977,2016-02-28,50110,77.83,1,1,1,1,...,1.0,2000,6,0.286,1,1,3,10,1,75358


## Dice

In [ ]:
dice_df_1 = df_com_bulid.loc[(df_com_bulid['district'] == 7) &
                    (df_com_bulid['square'] > 120)]
dice_df_1

,surrogate_keys,id,cid,trade_time,price,square,living_room,drawing_room,kitchen,bath_room,...,building_type,construction_time,building_structure,ladder_ratio,five_years_property,elevator,renovation_condition,district,subway,community_average
0,1,101084782030,1111027376244,2016-08-09,31680,131.00,2,1,1,1,...,1.0,2005,6,0.217,0,1,3,7,1,56021
86,2,101086012217,1111027381879,2016-07-28,43436,132.38,2,2,1,2,...,1.0,2004,6,0.667,1,1,4,7,0,71539
87,8,101087303800,1111027381879,2016-09-07,49521,161.55,3,2,1,2,...,1.0,2004,6,0.667,1,1,4,7,0,71539
88,1996,101091724353,1111027381879,2016-08-11,61169,145.50,3,2,1,2,...,4.0,2004,6,0.500,1,1,1,7,0,71539
91,121811,BJCY27997757,1111027381879,2012-03-27,31727,132.38,2,1,1,2,...,1.0,2004,6,0.400,0,1,1,7,0,71539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317867,168814,BJCY89467233,1111027382232,2015-06-26,64550,458.56,5,4,1,4,...,0.0,2011,6,1.000,0,1,4,7,0,114246
317868,176562,BJCY90300669,1111027382232,2016-03-08,75753,574.24,5,3,2,7,...,0.0,2011,6,1.000,1,1,4,7,0,114246
317869,167207,BJCY89320665,1111055406217,2015-06-06,53319,398.55,5,3,1,5,...,0.0,2015,6,1.000,0,1,2,7,1,87646
317879,176595,BJCY90308700,1111027375368,2015-12-23,29262,922.70,3,1,1,1,...,4.0,0,6,0.118,1,1,3,7,1,59015


## Combinations

1. find total number of entries in each building type in district seven

In [ ]:
building_types = slice_df_1.groupby(['building_type']).agg({'id':'count'}).reset_index().rename(columns={'id': 'count'})
building_types


,building_type,count
0,0.0,382
1,1.0,36338
2,3.0,22629
3,4.0,47894


In [ ]:
df_district_sale = rollup_df2.loc[rollup_df2['district'] == 7]
df_district_sale

,district,building_count
6,7,107243
